In [1]:
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from pyxirr import xirr
pd.options.display.float_format = '{:,.2f}'.format


# Imputs

In [2]:
valor_imovel=720000
entrada_pp=0.2
n_parcelas=420                      # 35 anos
juros_nominal=0.0799                # a.a

# Custos Iniciais

In [3]:
valor_entrada=valor_imovel*entrada_pp

custo_itbi=0.03*valor_imovel
custo_escritura=3000
custo_registro=2000
custo_avaliacao=4000

total_custos_iniciais=valor_entrada+custo_itbi+custo_escritura+custo_registro+custo_avaliacao


# Cálculo financiamento

In [4]:
juros_efetivo=((1+juros_nominal/12)**(12)-1)            # Efeito anual dos juros sobre juros mensais.  

valor_finaciado=valor_imovel-valor_entrada

Encargos_finaciamento=130

In [5]:
amortizacao=valor_finaciado/n_parcelas
saldo_devedor=valor_finaciado

# Cálculo do juros em função do mês
def parcela(n):
    saldo_devedor=valor_finaciado-(n-1)*amortizacao
    juros=saldo_devedor*juros_nominal/12
    return amortizacao+juros+Encargos_finaciamento

df=pd.DataFrame(columns=['parcela','saldo_devedor'],index=range(0,n_parcelas))


for i in range(0,n_parcelas+1):
    if i==0:
        df.loc[i,'saldo_devedor']=valor_finaciado-i*amortizacao
        df.loc[i,'parcela']=0
    else:

        df.loc[i,'saldo_devedor']=valor_finaciado-i*amortizacao
        df.loc[i,'parcela']=parcela(i)
df

,parcela,saldo_devedor
0,0,"576,000.00"
1,"5,336.63","574,628.57"
2,"5,327.50","573,257.14"
3,"5,318.37","571,885.71"
4,"5,309.23","570,514.29"
...,...,...
416,"1,547.09","5,485.71"
417,"1,537.95","4,114.29"
418,"1,528.82","2,742.86"
419,"1,519.69","1,371.43"


# Custos fixos

In [6]:
condominio=704
iptu=1530/12
luz=120

total_custos_fixos=condominio+iptu+luz

In [7]:
for i in range(0,len(df)):
    df.loc[i,'custos_fixos']=total_custos_fixos
    if i==0:
        df.loc[i,'custos_iniciais']=total_custos_iniciais
    else:
        df.loc[i,'custos_iniciais']=0
df

,parcela,saldo_devedor,custos_fixos,custos_iniciais
0,0,"576,000.00",951.50,"174,600.00"
1,"5,336.63","574,628.57",951.50,0.00
2,"5,327.50","573,257.14",951.50,0.00
3,"5,318.37","571,885.71",951.50,0.00
4,"5,309.23","570,514.29",951.50,0.00
...,...,...,...,...
416,"1,547.09","5,485.71",951.50,0.00
417,"1,537.95","4,114.29",951.50,0.00
418,"1,528.82","2,742.86",951.50,0.00
419,"1,519.69","1,371.43",951.50,0.00


# Custos com reforma

In [8]:
obra=95000
prazo_obra=4                # Prazo da obra em mesês

def reforma(i, n,total):
    if i==0:
        return 0
    elif i<=n:
        return total/n
    else:
        return 0

for i in range(0,len(df)):
    df.loc[i,'reforma']=reforma(i,prazo_obra,obra)
df

,parcela,saldo_devedor,custos_fixos,custos_iniciais,reforma
0,0,"576,000.00",951.50,"174,600.00",0.00
1,"5,336.63","574,628.57",951.50,0.00,"23,750.00"
2,"5,327.50","573,257.14",951.50,0.00,"23,750.00"
3,"5,318.37","571,885.71",951.50,0.00,"23,750.00"
4,"5,309.23","570,514.29",951.50,0.00,"23,750.00"
...,...,...,...,...,...
416,"1,547.09","5,485.71",951.50,0.00,0.00
417,"1,537.95","4,114.29",951.50,0.00,0.00
418,"1,528.82","2,742.86",951.50,0.00,0.00
419,"1,519.69","1,371.43",951.50,0.00,0.00


# Cenários de venda

In [9]:
### projeções de inflação e crescimento econômico para evolução dos preços de venda

In [10]:
valor_venda=909000
prazo_venda=prazo_obra+2                # Prazo da obra em mesês + venda
comissao=2.5/100

def venda(i, n,total):
    if i==n:
        print(f'Comissão: {comissao*valor_venda}')
        return -total*(1-comissao)
    else:
        return 0

for i in range(0,len(df)):
    df.loc[i,'venda']=venda(i,prazo_venda,valor_venda)
df['total']=df.sum(axis=1)
df['total']=df['total']-df['saldo_devedor']

# quitação do saldo devedor e anulação das despesas à partir da venda
for i in range(0,len(df)):
    if i>prazo_venda:
        df.loc[i,'total']=0
    elif i==prazo_venda:
        df.loc[i,'total']=df.loc[i,'total']+df.loc[i,'saldo_devedor']

lucro=-df['total'].sum()
print(f'Lucro: {lucro}')
    

Comissão: 22725.0
Lucro: 10360.271428571432


In [11]:
df.head(12)


,parcela,saldo_devedor,custos_fixos,custos_iniciais,reforma,venda,total
0,0,"576,000.00",951.50,"174,600.00",0.00,0.00,"175,551.50"
1,"5,336.63","574,628.57",951.50,0.00,"23,750.00",0.00,"30,038.13"
2,"5,327.50","573,257.14",951.50,0.00,"23,750.00",0.00,"30,029.00"
3,"5,318.37","571,885.71",951.50,0.00,"23,750.00",0.00,"30,019.87"
4,"5,309.23","570,514.29",951.50,0.00,"23,750.00",0.00,"30,010.73"
5,"5,300.10","569,142.86",951.50,0.00,0.00,0.00,"6,251.60"
6,"5,290.97","567,771.43",951.50,0.00,0.00,"-886,275.00","-312,261.10"
7,"5,281.84","566,400.00",951.50,0.00,0.00,0.00,0
8,"5,272.71","565,028.57",951.50,0.00,0.00,0.00,0
9,"5,263.58","563,657.14",951.50,0.00,0.00,0.00,0


In [12]:
delta = relativedelta(months=1)
delta_d = timedelta(days=1)
start_date = datetime.today().date()+15*delta_d

for i in range(0, len(df)):
    if i==0:
        df.loc[i,'data']=start_date
    else:
        df.loc[i,'data']=start_date+i*delta

df


,parcela,saldo_devedor,custos_fixos,custos_iniciais,reforma,venda,total,data
0,0,"576,000.00",951.50,"174,600.00",0.00,0.00,"175,551.50",2022-08-15
1,"5,336.63","574,628.57",951.50,0.00,"23,750.00",0.00,"30,038.13",2022-09-15
2,"5,327.50","573,257.14",951.50,0.00,"23,750.00",0.00,"30,029.00",2022-10-15
3,"5,318.37","571,885.71",951.50,0.00,"23,750.00",0.00,"30,019.87",2022-11-15
4,"5,309.23","570,514.29",951.50,0.00,"23,750.00",0.00,"30,010.73",2022-12-15
...,...,...,...,...,...,...,...,...
416,"1,547.09","5,485.71",951.50,0.00,0.00,0.00,0,2057-04-15
417,"1,537.95","4,114.29",951.50,0.00,0.00,0.00,0,2057-05-15
418,"1,528.82","2,742.86",951.50,0.00,0.00,0.00,0,2057-06-15
419,"1,519.69","1,371.43",951.50,0.00,0.00,0.00,0,2057-07-15


In [19]:
# calculo da IRR
def cal_irr(x):
    
    dates=list(x['data'])
    amounts=list(x['total'])
    t_aa=xirr(dates, amounts)
    t_am=(1+t_aa)**(1/12)-1
    print('TII a.a.(%): ',t_aa*100)
    print('TII a.m.(%): ',t_am*100)
    print('Soma dos valores: ',-x['total'].sum())
    return (t_aa*100,t_am*100,-x['total'].sum())

In [20]:
cal_irr(df)

TII a.a.(%):  8.518743977172852
TII a.m.(%):  0.6835986718487952
Soma dos valores:  10360.271428571432


(8.518743977172852, 0.6835986718487952, 10360.271428571432)